In [1]:
import concurrent
import gc
import logging
import multiprocessing
import os
import sys
from concurrent import futures
import time

import numpy as np
import pandas as pd
import pyarrow as pa
import scipy.sparse
import scipy.sparse
import tiledb
import tiledbsoma as soma
from somacore import ExperimentAxisQuery, AxisQuery
from tiledb import ZstdFilter, ArraySchema, Domain, Dim, Attr, FilterList

In [3]:
dataset_ids_to_query = [ '218acb0f-9f2f-4f76-b90b-15a4b7c7f629',]
dataset_query = '('
for idx, di in enumerate(dataset_ids_to_query): 
    dataset_query += f'dataset_id == "{di}" '
    if idx != len(dataset_ids_to_query)-1:
        dataset_query += 'or '
dataset_query += ')'

celltypes_to_query = [
    'conventional dendritic cell',
    'plasmacytoid dendritic cell',
    'conventional dendritic cell',
    'plasmacytoid dendritic cell, human',
    'dendritic cell',
    'dendritic cell, human',
    'myeloid dendritic cell',
    'plasmacytoid dendritic cell']
celltype_query = '('
for idx, ct in enumerate(celltypes_to_query): 
    celltype_query += f'cell_type == "{ct}" '
    if idx != len(celltypes_to_query)-1:
        celltype_query += 'or '
celltype_query += ')'

OBS_VALUE_FILTER_1 = celltype_query # All cells in three datasets
OBS_VALUE_FILTER_2 = celltype_query + ' and ' + celltype_query # only relevant celltypes

In [4]:
OBS_VALUE_FILTER_2

'(cell_type == "conventional dendritic cell" or cell_type == "plasmacytoid dendritic cell" or cell_type == "conventional dendritic cell" or cell_type == "plasmacytoid dendritic cell, human" or cell_type == "dendritic cell" or cell_type == "dendritic cell, human" or cell_type == "myeloid dendritic cell" or cell_type == "plasmacytoid dendritic cell" ) and (cell_type == "conventional dendritic cell" or cell_type == "plasmacytoid dendritic cell" or cell_type == "conventional dendritic cell" or cell_type == "plasmacytoid dendritic cell, human" or cell_type == "dendritic cell" or cell_type == "dendritic cell, human" or cell_type == "myeloid dendritic cell" or cell_type == "plasmacytoid dendritic cell" )'

In [7]:
# init multiprocessing
start = time.time()
if multiprocessing.get_start_method(True) != "spawn":
    multiprocessing.set_start_method("spawn", True)

exp_uri = 's3://cellxgene-data-public/cell-census/2023-10-30/soma/census_data/homo_sapiens'
layer = "raw"
measurement_name = "RNA"

with soma.Experiment.open(uri=exp_uri,
                          context=soma.SOMATileDBContext().replace(tiledb_config={
                              "vfs.s3.region":"us-west-2",
                              "vfs.s3.no_sign_request":True})
                          ) as exp:

    query = exp.axis_query(measurement_name=measurement_name,
                           obs_query=AxisQuery(value_filter=OBS_VALUE_FILTER_2),
                           # Note: Must use *all* genes to compute size factors correctly, even when var filter is
                           # being used for testing
                           var_query=AxisQuery())

In [8]:
TEST_MODE = bool(os.getenv("TEST_MODE", False))  # Read data from simple test fixture Census data
PROFILE_MODE = bool(os.getenv("PROFILE_MODE", False))  # Run pass 2 in single-process mode with profiling output

ESTIMATORS_CUBE_ARRAY_URI = "estimators_cube"

OBS_WITH_SIZE_FACTOR_TILEDB_ARRAY_URI = "obs_with_size_factor"

TILEDB_SOMA_BUFFER_BYTES = 2**31
if TEST_MODE:
    TILEDB_SOMA_BUFFER_BYTES = 10 * 1024 ** 2

# The minimum number of cells that should be processed at a time by each child process.
MIN_BATCH_SIZE = 2**14
# For testing
MIN_BATCH_SIZE = 20000

CUBE_TILEDB_DIMS_OBS = [
    "cell_type",
    "dataset_id",
]

CUBE_TILEDB_ATTRS_OBS = [
    "assay",
    "suspension_type",
    "donor_id",
    "disease",
    "sex"
]

if TEST_MODE:
    CUBE_TILEDB_DIMS_OBS = ["celltype"]
    CUBE_TILEDB_ATTRS_OBS = ["study"]

CUBE_LOGICAL_DIMS_OBS = CUBE_TILEDB_DIMS_OBS + CUBE_TILEDB_ATTRS_OBS

CUBE_DIMS_VAR = ['feature_id']

if TEST_MODE:
    CUBE_DIMS_VAR = ['var_id']

CUBE_TILEDB_DIMS = CUBE_DIMS_VAR + CUBE_TILEDB_DIMS_OBS

ESTIMATOR_NAMES = ['nnz', 'n_obs', 'min', 'max', 'sum', 'mean', 'sem', 'var', 'sev', 'selv']


CUBE_SCHEMA = ArraySchema(
  domain=Domain(*[
    Dim(name=dim_name, dtype="ascii", filters=FilterList([ZstdFilter(level=-1), ]))
    for dim_name in CUBE_TILEDB_DIMS
  ]),
  attrs=[Attr(name=attr_name, dtype='ascii', nullable=False, filters=FilterList([ZstdFilter(level=-1), ]))
         for attr_name in CUBE_TILEDB_ATTRS_OBS] +
        [Attr(name=estimator_name, dtype='float64', var=False, nullable=False, filters=FilterList([ZstdFilter(level=-1), ]))
         for estimator_name in ESTIMATOR_NAMES],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)


In [9]:
obs_df = query.obs().concat().to_pandas()
var_df = query.var().concat().to_pandas()

In [13]:
X

<bound method ExperimentAxisQuery.X of <somacore.query.query.ExperimentAxisQuery object at 0x7fd5667a89d0>>

In [ ]:
adata = query.to_anndata('raw')

In [15]:
X

In [10]:
counts = obs_df.groupby(['dataset_id', 'cell_type']).size().reset_index(name='count')

In [11]:
counts

,dataset_id,cell_type,count
0,00ff600e-6e2e-4d76-846f-0eec4f0ae417,conventional dendritic cell,28
1,00ff600e-6e2e-4d76-846f-0eec4f0ae417,plasmacytoid dendritic cell,117
2,01ad3cd7-3929-4654-84c0-6db05bd5fd59,conventional dendritic cell,7895
3,01ad3cd7-3929-4654-84c0-6db05bd5fd59,plasmacytoid dendritic cell,2224
4,055ca631-6ffb-40de-815e-b931e10718c0,conventional dendritic cell,451
...,...,...,...
212,fa8605cf-f27e-44af-ac2a-476bee4410d3,plasmacytoid dendritic cell,575
213,fd072bc3-2dfb-46f8-b4e3-467cb3223182,dendritic cell,19224
214,fd072bc3-2dfb-46f8-b4e3-467cb3223182,plasmacytoid dendritic cell,3005
215,fe52003e-1460-4a65-a213-2bb1a508332f,conventional dendritic cell,1503


In [67]:
counts = obs_df.groupby(['dataset_id', 'cell_type']).size().reset_index(name='count')

In [68]:
counts.query('dataset_id == "218acb0f-9f2f-4f76-b90b-15a4b7c7f629"')

,dataset_id,cell_type,count
0,218acb0f-9f2f-4f76-b90b-15a4b7c7f629,conventional dendritic cell,18203
1,218acb0f-9f2f-4f76-b90b-15a4b7c7f629,plasmacytoid dendritic cell,5233
